In [1]:
from sklearn.preprocessing import StandardScaler
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Sequential
from keras.layers import LSTM, Dense ,Dropout, Bidirectional, Input
from keras.optimizers import Nadam
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import ParameterGrid
from tqdm import tqdm
from neural_network_functions import *

In [20]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [3]:
model_params = ParameterGrid({"units": [15, 30, 60, 90, 150],
                              "n_timesteps": [15, 30, 60],
                              "optimizer": ['adam', 'rmsprop', 'nadam'],
                              "loss": ['mean_squared_error', 'mean_absolute_error'],
                              "epochs" : [100, 250, 500],
                              "batch_size": [32, 64]})

In [4]:
scaler = MinMaxScaler(feature_range=(0,1))

In [5]:
df = pd.read_csv('dataset_days.csv', sep=',', index_col="time")
df.head()

Среднняя t  Минимальная t  Максимальная t  Суточная сумма осадков  \
time                                                                            
2019-05-24        13.6            8.9            17.5                     0.0   
2019-05-25        15.6           11.8            18.9                     0.3   
2019-05-26        18.1           14.3            23.6                     0.0   
2019-05-27        16.7           12.5            21.2                     0.0   
2019-05-28        22.0           15.9            28.4                     0.0   

            Глубина снега  Скорость ветра  Среднее давление hPa  \
time                                                              
2019-05-24            0.0             4.7                1012.7   
2019-05-25            0.0             4.6                1010.2   
2019-05-26            0.0             2.4                1004.7   
2019-05-27            0.0             4.7                1008.5   
2019-05-28            0.0             2.9                1014.1   

            Относительная влажность в %  
time                                     
2019-05-24                       64.125  
2019-05-25                       56.625  
2019-05-26                       62.375  
2019-05-27                       52.500  
2019-05-28                       48.250

In [6]:
temp = df.iloc[:,:1].values
train, test, size = create_train_test_data(temp, 70)

In [7]:
train_scal = scaler.fit_transform(train)
test_scal = scaler.transform(test)

In [22]:
n_features = 1
n_forecast = 10

In [18]:
# d = {"params": [],
# "day1": [],
# "day2": [],
# "day3": [],
# "day4": [],
# "day5": [],
# "day6": [],
# "day7": [],
# "day8": [],
# "day9": [],
# "day10": []}

# df = pd.DataFrame(d)

# df.to_csv("res_test_model.csv", index=False)

In [19]:
dataset = pd.read_csv('res_test_model.csv', sep=',')
dataset.head()

Empty DataFrame
Columns: [params, day1, day2, day3, day4, day5, day6, day7, day8, day9, day10]
Index: []

In [ ]:
for item in tqdm(model_params):

    d = {"params": [],
         "day1": [],
         "day2": [],
         "day3": [],
         "day4": [],
         "day5": [],
         "day6": [],
         "day7": [],
         "day8": [],
         "day9": [],
         "day10": []}
    
    x_train, y_train = create_data(train_scal, item["n_timesteps"], n_forecast, n_features)
    x_test, y_test = create_data(test_scal, item["n_timesteps"], n_forecast, n_features)
    y_test = np.reshape(y_test, (y_test.shape[0], y_test.shape[1]))
    y_test = scaler.inverse_transform(y_test)

    model = create_model(x_train=x_train,
                        n_forecast=n_forecast,
                        units=item["units"],
                        n_features = n_features,
                        n_timesteps=item["n_timesteps"],
                        optimizer=item["optimizer"],
                        loss=item["loss"])
    
    model.fit(x_train, y_train, epochs=item["epochs"], batch_size=item["batch_size"], verbose=0)

    predict = model.predict(x_test)

    predict = np.reshape(predict, (predict.shape[0], predict.shape[1]))
    predict = scaler.inverse_transform(predict)

    d["params"].append(item)
    for i in range(n_forecast):
        mae_day = np.mean(np.abs(y_test[:, i] - predict[:, i]))
        d[f"day{i+1}"].append(mae_day)

    df_item = pd.DataFrame(d)
    dataset = pd.concat([dataset, df_item], axis=0)
    dataset_sort = dataset.sort_values(by=["day1", "day2", "day3", "day4", "day5", "day6", "day7", "day8", "day9", "day10"])
    dataset_sort.to_csv('res_test_model.csv', index=False)

In [ ]:
dataset['mean'] = dataset[['day1', 'day2', 'day3', 'day4', 'day5', 'day6', 'day7', 'day8', 'day9', 'day10']].mean(axis=1)

In [ ]:
dataset.head()

In [ ]:
dataset.to_csv("res_test_model.csv", index=False)